In [11]:
import cv2
import cvlib as cv
import numpy as np
from keras.models import load_model

In [12]:
def getClassName(classIndex):
    "A basic function to get the class name."
    if classIndex==0:
        return "Sad"
    
    elif classIndex==1:
        return "Shocked"
    
    elif classIndex==2:
        return "Happy"

In [13]:
model = load_model("final_model.h5") # Loading the pretrained model to predict

***

In [14]:
cap = cv2.VideoCapture(0)
# Above^ Class instance for video capturing.
# If a video name is entered, captures the specified video.

threshold = 0.80 * 100

while cap.isOpened(): # Repeats until the key "q" is pressed.
    
    ret, frame = cap.read() # Starts capturing. 
    # ret is True or False to represent if capturing is successful.
    # frame is the image to process.
    if not ret:
        print("Can't receive the frame.")
        break
        
    # FACE RECOGNITION    
    # Note: input image (numpy array, BGR order, dtype - uint8)
    faces, confidences = cv.detect_face(frame, threshold=0.70)
    
    
    for _, f in enumerate(faces):
        # corner points: 
        (startX, startY) = (f[0], f[1])    # top left corner
        (endX, endY) = (f[2], f[3])    # bottom right corner

        cropped_frame = np.copy(frame[startY:endY, startX:endX])

        if (cropped_frame.shape[0]) < 10 or (cropped_frame.shape[1]) < 10:
            continue
    
        # preprocessing on the cropped frame
        cropped_frame = cv2.resize(cropped_frame, (64,64))
        cropped_frame = cropped_frame.astype("float64") / 255.0
        cropped_frame = np.expand_dims(cropped_frame, axis=0)
        confidences = model.predict(cropped_frame)[0]    # Probability values for all classes
        max_probability = max(confidences)*100
        classIndex = np.argmax(confidences)
        className = getClassName(classIndex)

        # different colors for different sentiments
        if max_probability > threshold:
            frame_text = f"Sentiment: {className} ({max_probability:.2f}%)"
            if className == "Shocked":
                rect_color = text_color = (0, 255, 255)
            elif className == "Sad":
                rect_color = text_color = (255, 255, 0)
            else:
                rect_color = text_color = (0, 255, 0)
        else:
            rect_color = text_color = (0, 255, 0)
            frame_text = "Reading..."    
        
        cv2.rectangle(img=frame, 
                      pt1=(startX, startY), 
                      pt2=(endX, endY), 
                      color=rect_color, 
                      thickness=2)
        
        startY = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.putText(img=frame, 
                    text=frame_text, 
                    org=(startX, startY), 
                    fontFace=cv2.FONT_HERSHEY_TRIPLEX, 
                    fontScale=0.6, 
                    color=text_color, 
                    thickness=1)  
    
    cv2.imshow("Face_Sentiment", frame)
    #cv2.imwrite("saved_picture.jpg", frame)
    if cv2.waitKey(1) > 0:    # Repeats until a key is pressed.
        break

cap.release() # Releases the camera after a key is pressed
cv2.destroyWindow("Face_Sentiment") # Closes the window named "Face_Sentiment"
# cv2.waitKey() # Necessary for not having trouble with the closing window

In [15]:
cap.release()
cv2.destroyAllWindows()